In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('mmc4.xlsx')

In [3]:
import pyensembl 
genome = pyensembl.ensembl_grch38

In [4]:
seqs = genome.protein_sequences.fasta_dictionary

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


In [5]:
pep_lengths = set(df.PEP_LEN)

In [6]:
pep_lengths

{8, 9, 10, 11, 12, 13, 14}

In [7]:
from collections import defaultdict
from tqdm import tqdm
kmers_by_size = {}
kmer_to_source = defaultdict(set)
for k in pep_lengths:
    curr_kmers = []
    for protein_id, seq in tqdm(seqs.items()):
        n = len(seq)
        if n < k:
            continue
    
        for i in range(n - k + 1):
            kmer = seq[i:i+k]
            curr_kmers.append(kmer)
            kmer_to_source[kmer].add(protein_id)
    kmers_by_size[k] = set(curr_kmers)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121766/121766 [02:36<00:00, 779.09it/s]


In [43]:
import numpy as np
aas = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
assert len(aas) == 20
ref_peptides_col = []
for _, row in df.iterrows():
    alt_peptide = row.ALT_EPI_SEQ
    
    pos = row.MUTATION_POSITION
    ref_peptides = set()
    if not np.isnan(pos):
        pos = int(pos)
        i = pos - 1
        alt_aa = alt_peptide[i]
        
        for aa in aas:
            if aa == alt_aa:
                continue
            peptide = alt_peptide[:i] + aa + alt_peptide[i + 1:]
            if len(kmer_to_source[peptide]) > 0:
                ref_peptides.add(peptide)
        if len(ref_peptides) == 0:
            for aa1 in aas:
                for aa2 in aas:
                    peptide = alt_peptide[:i] + aa1 + aa2 + alt_peptide[i + 2:]
                    
                    if peptide == alt_peptide:
                        continue
                    if len(kmer_to_source[peptide]) > 0:
                        print("MNV?", alt_peptide, aa1, aa2, aa3, peptide)
                        ref_peptides.add(peptide)
    ref_peptides = list(ref_peptides)
    ref_peptides_col.append(";".join(ref_peptides))

MNV? RRHIEIRDK D Q Y RRHIEIRDQ


In [44]:
df["ref_peptide_candidates"] = ref_peptides_col

In [45]:
df.ref_peptide_candidates.str.contains(";").sum()

16

In [47]:
(df.ref_peptide_candidates.str.len() == 0).sum()

34

In [48]:
df.to_csv("tesla4_with_inferred_ref_peptides.csv")

In [49]:
len(df)

608

In [52]:
len(df) - ((df.ref_peptide_candidates.str.len() == 0).sum() + df.ref_peptide_candidates.str.contains(";").sum())

558